In [2]:
import pandas as pd
import jieba
from gensim.models import Word2Vec

# 加载数据
data = pd.read_csv('train.csv')
print("数据样例：\n", data.head(2))

数据样例：
    label             comment
0      0  一如既往地好吃，希望可以开到其他城市
1      0   味道很不错，分量足，客人很多，满意


In [3]:
# 定义预处理函数
def preprocess(text):
    text = str(text)
    # 去除标点符号
    text = text.replace("，", "").replace("!", "").replace("！", "").replace("。", "") \
              .replace("~", "").replace("；", "").replace("？", "").replace("?", "") \
              .replace("【", "").replace("】", "").replace("#", "")
    # 精确分词
    return list(jieba.cut(text))

# 对评论文本进行分词处理
corpus = data['comment'].apply(preprocess)
print("分词样例：", corpus[0])

分词样例： ['一如既往', '地', '好吃', '希望', '可以', '开', '到', '其他', '城市']


In [4]:
# 训练Skip-Gram模型（sg=1）
model = Word2Vec(
    sentences=corpus,
    vector_size=100,  # 词向量维度
    window=5,         # 上下文窗口大小
    min_count=1,      # 最小词频
    sg=1,             # Skip-Gram模式
    workers=4         # 并行线程数
)
print("模型训练完成！")

模型训练完成！


In [5]:
try:
    vector = model.wv["环境"]
    print("词向量（前5维）：", vector[:5])
    print("形状：", vector.shape)
except KeyError:
    print("词语 '环境' 不在词汇表中！")

词向量（前5维）： [ 0.1254612   0.473996    0.00405439  0.17687885 -0.02791959]
形状： (100,)


In [6]:
try:
    similar_words = model.wv.most_similar("好吃", topn=3)
    for word, score in similar_words:
        print(f"{word}: {score:.4f}")
except KeyError:
    print("词语 '好吃' 不在词汇表中！")

入味: 0.8962
好看: 0.8819
美味: 0.8747


In [7]:
def safe_similarity(word1, word2):
    if word1 in model.wv and word2 in model.wv:
        return model.wv.similarity(word1, word2)
    else:
        return "词语不存在！"

sim1 = safe_similarity("好吃", "美味")
sim2 = safe_similarity("好吃", "蟑螂")
print(f"好吃 vs 美味: {sim1:.4f}")
print(f"好吃 vs 蟑螂: {sim2:.4f}")

好吃 vs 美味: 0.8747
好吃 vs 蟑螂: 0.5082


In [8]:
try:
    result = model.wv.most_similar(
        positive=["餐厅", "聚会"],
        negative=["安静"],
        topn=1
    )
    print("餐厅 + 聚会 - 安静 =", result[0][0])
except KeyError as e:
    print(f"词语 {e} 不在词汇表中！")

餐厅 + 聚会 - 安静 = 客户
